## Google Distance Matrix Trial
Tested Google Distance Matrix based on 10 O-D pairs with different departure times(durtion, duration_in_traffic).

In [ ]:
import requests
import googlemaps
import datetime
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import itertools
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import json
import folium
import polyline
from shapely import wkt
import time

In [ ]:
# Jessica Google API Key
key = 'Jessica Google API Key'

In [ ]:
origins_df = pd.read_csv("../Data/origins_gdf.csv")
destinations_df = pd.read_csv("../Data/destinations_gdf.csv")

In [ ]:
origins_df['geometry'] = origins_df['geometry'].apply(wkt.loads)
destinations_df['geometry'] = destinations_df['geometry'].apply(wkt.loads)

In [ ]:
origins_gdf = gpd.GeoDataFrame(origins_df, geometry='geometry')
destinations_gdf = gpd.GeoDataFrame(destinations_df, geometry='geometry')

In [ ]:
type(destinations_gdf)

## Calculate Free-Flow Travel Time

In [ ]:
gmaps = googlemaps.Client(key=key)

In [ ]:
# extract origin and destination coordinates from the GeoDataFrames
origins = origins_gdf.geometry.apply(lambda point: (point.y, point.x)).tolist()
destinations = destinations_gdf.geometry.apply(lambda point: (point.y, point.x)).tolist()

In [ ]:
# trial at 9/10 6:35 pm
result = gmaps.distance_matrix(origins, destinations, mode="driving")

In [ ]:
result

In [ ]:
# create free_flow travel time (at 9/8 10:48 pm)
free_flow_times = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving")

    # extract travel time from the result
    travel_time = result['rows'][0]['elements'][0]['duration']['text']
    free_flow_times.append(travel_time)

In [ ]:
free_flow_times

In [ ]:
# create free_flow travel time (at 9/10 7/52 pm)
free_flow_times = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving")

    # extract travel time from the result
    travel_time = result['rows'][0]['elements'][0]['duration']['text']
    free_flow_times.append(travel_time)

In [ ]:
free_flow_times

In [ ]:
# create distance
distances = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving")

    # extract distance from the result
    distance = result['rows'][0]['elements'][0]['distance']['text']
    distances.append(distance)


In [ ]:
distances

## Real-Time Travel Time

In [ ]:
# trial at 9/10 6:35 pm
result = gmaps.distance_matrix(origins[0], destinations[0], mode="driving", departure_time='now')
result

In [ ]:
# create free_flow travel time (at 10:48 pm)
real_times = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving", departure_time='now')

    # extract travel time from the result
    travel_time = result['rows'][0]['elements'][0]['duration']['text']
    real_times.append(travel_time)

In [ ]:
real_times

In [ ]:
# create free_flow travel time (at 9/10 7:48 pm)
real_times = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving", departure_time='now')

    # extract travel time from the result
    travel_time = result['rows'][0]['elements'][0]['duration']['text']
    real_times.append(travel_time)

In [ ]:
real_times

In [ ]:
real_times = pd.DataFrame(real_times)
real_times

In [ ]:
# create travel time (at 9/10 7:48 pm)
traffic_times = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving", departure_time='now')

    # extract duration_in_traffic from the result
    travel_time = result['rows'][0]['elements'][0]['duration_in_traffic']['text']
    traffic_times.append(travel_time)

In [ ]:
traffic_times

In [ ]:
traffic_times = pd.DataFrame(traffic_times)
traffic_times

## Organize Data

In [ ]:
origins_df = origins_df.columns = ["origins_" + col for col in origins_df.columns]
destinations_df = destinations_df.columns = ["destinations_" + col for col in destinations_df.columns]

In [ ]:
combined_df = pd.concat([origins_df, destinations_df], axis=1)

In [ ]:
combined_df

In [ ]:
combined_df['free_flow_time_10:48pm'] = free_flow_times

In [ ]:
combined_df['real_times_10:48pm'] = real_times

In [ ]:
combined_df

## Set Times

In [ ]:
# trial

departure_time_obj = datetime.datetime(2023, 10, 1, 11)
departure_time_unix = int(time.mktime(departure_time_obj.timetuple()))

result = gmaps.distance_matrix(origins=origins[0], destinations=destinations[0], mode="driving", departure_time=departure_time_unix)

result

In [ ]:
# set departure_times
start_time = datetime.datetime(2023, 10, 1)
departure_times = [start_time + datetime.timedelta(hours=i) for i in range(24)]

In [ ]:
travel_times = []

# calculate the travel time for each pair
for i in range(10): 
    origin = origins[i]
    destination = destinations[i]
    
    travel_times_by_hour = {}

    for dt in departure_times:
        hour_str = dt.strftime('%H')
        column_name = f'duration_{hour_str}'
        
        result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving", departure_time=dt)
        
        travel_time = result['rows'][0]['elements'][0]['duration']['text']
        travel_times_by_hour[column_name] = travel_time
    
    travel_times.append(travel_times_by_hour)

In [ ]:
travel_times_df = pd.DataFrame(travel_times)

In [ ]:
travel_times_df

In [ ]:
travel_times_t = []

# calculate the travel time for each pair
for i in range(10):
    origin = origins[i]
    destination = destinations[i]
    
    travel_times_by_hour = {}

    for dt in departure_times:
        hour_str = dt.strftime('%H')
        column_name = f'duration_in_traffic_{hour_str}'
        
        result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving", departure_time=dt)
        
        travel_time = result['rows'][0]['elements'][0]['duration_in_traffic']['text']
        travel_times_by_hour[column_name] = travel_time
    
    travel_times_t.append(travel_times_by_hour)

In [ ]:
travel_times_df2 = pd.DataFrame(travel_times_t)

In [ ]:
travel_times_df2

## Organize Data

In [ ]:
combined_df = pd.concat([combined_df, travel_times_df], axis=1)

In [ ]:
combined_df = pd.concat([combined_df, travel_times_df2], axis=1)

In [ ]:
combined_df['distance'] = distances

In [ ]:
combined_df